In [2]:
import pandas as pd
from datetime import datetime, date, timedelta
import numpy as np

In [26]:
# This was a tester from the blog post to figure out how to calculate beta.

x0 = np.array([35240, 45071, 55674, 69660, 86570, 105363, 125415])
x1 = np.array([45071, 55674, 69660, 86570, 105363, 125415, 143444])
active_cases = np.array([34100, 43374, 53468, 66488, 82439, 100022, 118522])
cum_rec = np.array([668, 1096, 1400, 2107, 2807, 3612, 4685])
cum_dead = np.array([472, 601, 806, 1065, 1324, 1729, 2208])
growth_rate = (x1 - x0) / x0
#print(growth_rate)
doubling_time = 1.0 / np.log2(growth_rate + 1)

gamma = 1/14
beta = growth_rate + gamma

for val in beta:
    print(val)

0.3504013296578563
0.3066796197745145
0.32264098655951223
0.31417907386899635
0.28851301176587074
0.26174205908552883
0.21518330571075459


In [38]:
# la, nyc, slc county data from nytimes county level data
la = pd.read_csv("la_data.csv")
nyc = pd.read_csv("la_data.csv")
slc = pd.read_csv("la_data.csv")

In [43]:
# add admits to data for LA county.
la["admits"] = la.cases - la.cases.shift()
# Pretend customer gave us rows 60-65
la_train = la.iloc[60:66]
la_train = la_train[["date", "cases", "admits"]]
# Use rows 66-end to project
la_test = la.iloc[66:]
la_test = la_test[["date", "cases", "admits"]]

In [44]:
la_train

,date,cases,admits
60,2020-03-26,1216,417.0
61,2020-03-27,1465,249.0
62,2020-03-28,1804,339.0
63,2020-03-29,2136,332.0
64,2020-03-30,2474,338.0
65,2020-03-31,3011,537.0


In [46]:
gamma = 1/14

growth_rate = la_train.admits / la_train.cases
doubling_time = 1.0 / np.log2(growth_rate + 1)
beta = growth_rate + gamma

# Print doubling time and beta from each time step
for val1, val2 in zip(doubling_time, beta):
    print("doubling time: ", val1)
    print("beta: ", val2)
    print(" ")

doubling time:  2.3508306166837216
beta:  0.41435620300751874
 
doubling time:  4.415665203162323
beta:  0.24139444173573865
 
doubling time:  4.025238058736619
beta:  0.25934431422236304
 
doubling time:  4.797756627452325
beta:  0.2268592830390583
 
doubling time:  5.4126878376129435
beta:  0.208049428340455
 
doubling time:  4.223627535779114
beta:  0.2497746358589932
 


In [ ]:
# Got stuck here...
population = 13000000
market_share = 100
la_train["i"] = "wha??"

In [28]:
# Just scratchwork below.

# Doesn't change
population = 328200000
# Varies with time
for i0, r_act, cd in zip(x0, cum_rec, cum_dead):
    r_calc = i0 * 1/14
    s_calc = population - i0 - r_calc
    print("s: ", s_calc)
    print("i: ", i0)
    print("rc: ", r_calc)
    print("r_act: ", r_act+cd)
    print("  ")

s:  328162242.85714287
i:  35240
rc:  2517.1428571428573
r_act:  1140
  
s:  328151709.64285713
i:  45071
rc:  3219.3571428571427
r_act:  1697
  
s:  328140349.28571427
i:  55674
rc:  3976.714285714286
r_act:  2206
  
s:  328125364.28571427
i:  69660
rc:  4975.714285714285
r_act:  3172
  
s:  328107246.4285714
i:  86570
rc:  6183.571428571428
r_act:  4131
  
s:  328087111.0714286
i:  105363
rc:  7525.928571428572
r_act:  5341
  
s:  328065626.78571427
i:  125415
rc:  8958.214285714286
r_act:  6893
  


In [49]:
def sir(
    s: float, i: float, r: float, beta: float, gamma: float, n: float
) -> Tuple[float, float, float]:
    """The SIR model, one time step."""
    s_n = (-beta * s * i) + s
    i_n = (beta * s * i - gamma * i) + i
    r_n = gamma * i + r
    scale = n / (s_n + i_n + r_n)
    return s_n * scale, i_n * scale, r_n * scale


def sim_sir(
    s: float, i: float, r: float, gamma: float, i_day: int, beta: np.array
):
    """Simulate SIR model forward in time, returning a dictionary of daily arrays
    Parameter order has changed to allow multiple (beta, n_days)
    to reflect multiple changing social distancing policies.
    """
    s, i, r = (float(v) for v in (s, i, r))
    n = s + i + r
    d = i_day
    

    s_a = np.empty(n_days, "float")
    i_a = np.empty(n_days, "float")
    r_a = np.empty(n_days, "float")
    beta_a = np.empty(n_days, "float")

    index = 0
    for _ in range(n_days):
        beta = 
        d_a[index] = d
        s_a[index] = s
        i_a[index] = i
        r_a[index] = r
        index += 1

        s, i, r = sir(s, i, r, beta, gamma, n)
        d += 1

    d_a[index] = d
    s_a[index] = s
    i_a[index] = i
    r_a[index] = r
    return {
        "day": d_a,
        "susceptible": s_a,
        "infected": i_a,
        "recovered": r_a,
        "ever_infected": i_a + r_a
    }


def build_sim_sir_w_date_df(
    raw_df: pd.DataFrame,
    current_date: datetime,
    keys: Sequence[str],
) -> pd.DataFrame:
    day = raw_df.day
    return pd.DataFrame({
        "day": day,
        "date": day.astype('timedelta64[D]') + np.datetime64(current_date),
        **{
            key: raw_df[key]
            for key in keys
        }
    })

sim_sir()